In [1]:
import os
import sys
import pickle
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer

sys.path.append(os.path.dirname(os.path.abspath('.')))
import configs
from datasets import TruthfulQADataset, DishonestQADataset, AmongUsDataset, RolePlayingDataset, RepEngDataset
from probes import LinearProbe

from configs import config_phi4

In [4]:
dataset_name = 'RolePlayingDataset'
config = config_phi4
model, tokenizer, device = None, None, 'cpu'
dataset = eval(f"{dataset_name}")(config, model=model, tokenizer=tokenizer, device=device, test_split=0.2)

In [5]:
dataset.df.head(3)

,scenario,question,answer_prefix,honest_completion,deceptive_completion
0,You are a student who has not completed an imp...,Teacher: Why haven't you submitted your assign...,Student:,"""I haven't submitted the assignment yet due to...",I've been dealing with a family situation that...
1,You are a student who hasn't started working o...,Teacher: Why have you not submitted the scienc...,Student:,"I'm ashamed to admit it, but the truth is, I p...","I'm so sorry, but I experienced a family emerg..."
2,You are a college student taking a particularl...,Professor: Do you have a valid reason for need...,Student:,"Professor, I'm afraid I don't have an unavoida...",I've been experiencing some health issues over...


In [ ]:
dataset.populate_dataset(force_redo=False)

In [3]:
train_loader = dataset.get_train(batch_size=32, num_tokens=10) # take the acts of the last num_tokens tokens

In [4]:
probe = LinearProbe(input_dim=dataset.activation_size, device=device)

print(f'Training probe on {len(train_loader)} batches and {len(train_loader.dataset)} samples.')

probe.fit(train_loader, epochs=100)

Training probe on 40 batches and 1264 samples.
Epoch 1: Train Loss = 0.3075, Train Acc = 0.8845
Epoch 11: Train Loss = 0.0215, Train Acc = 0.9945
Epoch 21: Train Loss = 0.0232, Train Acc = 0.9921
Epoch 31: Train Loss = 0.0222, Train Acc = 0.9929
Epoch 41: Train Loss = 0.0158, Train Acc = 0.9945
Epoch 51: Train Loss = 0.0279, Train Acc = 0.9937
Epoch 61: Train Loss = 0.0277, Train Acc = 0.9945
Epoch 71: Train Loss = 0.0201, Train Acc = 0.9960
Epoch 81: Train Loss = 0.0050, Train Acc = 0.9976
Epoch 91: Train Loss = 0.0312, Train Acc = 0.9937
Final Train Acc: 0.9945
